## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [8]:
###Drop Nan Values
df=df.dropna()


In [9]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [10]:
## Get the Dependent features
y=df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.3.1'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [16]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Smarak
[nltk_data]     Pradhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
lemma=WordNetLemmatizer()

def preprocess(doc):
  doc=re.sub(r'\W', ' ',str(doc))
  doc = doc.lower()                 # Converting to lowercase
  cleanr = re.compile('<.*?>')
  doc = re.sub(cleanr, ' ',str(doc))        #Removing HTML tags
  doc = re.sub(r'[?|!|\'|"|#]',r'',str(doc))
  doc = re.sub(r'[.|,|)|(|\|/]',r' ',str(doc))
  doc=re.sub(r'\s+', ' ',str(doc),flags=re.I)
  doc=re.sub(r'^b\s+', ' ',str(doc))
  doc = re.sub(r'\[[0-9]*\]', ' ', doc)
  doc = re.sub(r'\s+', ' ',doc)
  # Removing special characters and digits
  doc = re.sub('[^a-zA-Z]', ' ', doc )
  doc = re.sub(r'\s+', ' ', doc)
  #doc_list = nltk.sent_tokenize(doc)
  stopwords = nltk.corpus.stopwords.words('english')
  #Lemmatization
  tokens=doc.split()
  tokens=[lemma.lemmatize(word) for word in tokens]
  tokens=[word for word in tokens if word not in stopwords]
  
  return tokens
corpus=[]
for i in range(0,len(messages)) :
    tokens=preprocess(messages['title'][i])
    tokens=' '.join(tokens)
    corpus.append(tokens)

In [38]:
onehot_repr=[one_hot(words,voc_size)for words i

8, 856, 4489, 168, 1766],
 [1203, 3558, 1231, 1801, 1285, 1206],
 [1658, 3182, 922, 1906, 2306, 4489, 1804, 4054, 3913, 3990, 13, 274],
 [1494, 94, 3130, 3848],
 [4104, 20, 1836, 4586, 3980, 3044, 455, 4884],
 [157, 1517, 3441, 545, 1508, 1187, 2278, 4489, 168, 1766],
 [572, 2908, 3337, 206, 4028, 3635, 4652, 2712],
 [2628, 2776, 2628, 2865, 3162, 3050],
 [2023, 2282, 1841, 3453, 2272, 2428, 1914, 592, 2780, 4489, 168, 1766],
 [3850, 1518, 2418, 2151, 739, 3611, 3181, 2006, 1313, 4489, 168, 1766],
 [518, 5, 335, 3231, 600, 3154, 4925, 3139],
 [719, 2130, 2418, 4505, 93, 1433, 2482, 4228, 4489, 168, 1766],
 [1498, 3284, 850, 2684, 492],
 [2848, 4201, 1752, 2168, 3921, 3437, 4240, 821, 3297, 4489, 168, 1766],
 [525, 804, 948, 3561, 1759, 2574, 1605, 3934, 4301],
 [3412, 4436, 4553, 2981, 584, 4992, 1494],
 [137, 1906, 2401, 3299, 1494, 4476, 4869, 1313],
 [4930, 2272, 834, 2201, 4339, 4930],
 [1494, 1608, 4980, 801],
 [3088, 4639, 2316, 1605, 1626, 4275, 3115, 457, 2712, 4930],
 [4653, 3

### Embedding Representation

In [40]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1670 1257 4909]
 [   0    0    0 ...  108 2392 4930]
 [   0    0    0 ... 4909 3799 3551]
 ...
 [   0    0    0 ... 4489  168 1766]
 [   0    0    0 ... 1717  491  748]
 [   0    0    0 ... 1504 4804  173]]


In [41]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 1020, 1954, 3694, 4413, 4573, 4017, 2721,
       1670, 1257, 4909])

In [42]:
## Creating model
embedding_vector_features=100
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 100)           500000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 580,501
Trainable params: 580,501
Non-trainable params: 0
_________________________________________________________________
None


In [43]:
len(embedded_docs),y.shape

(18285, (18285,))

In [56]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [57]:
X_final.shape,y_final.shape

((18285, 25), (18285,))

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [59]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 27ms/step - loss: 0.3476 - accuracy: 0.8398 - val_loss: 0.1982 - val_accuracy: 0.9183
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1500 - accuracy: 0.9414 - val_loss: 0.1909 - val_accuracy: 0.9221
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1072 - accuracy: 0.9611 - val_loss: 0.2003 - val_accuracy: 0.9168
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0813 - accuracy: 0.9705 - val_loss: 0.2312 - val_accuracy: 0.9157
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0615 - accuracy: 0.9762 - val_loss: 0.3032 - val_accuracy: 0.9145
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0449 - accuracy: 0.9848 - val_loss: 0.3038 - val_accuracy: 0.9087
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0351 - accuracy: 0.9879 - val_loss: 0.3664 - val_accuracy: 0.9160

### Adding Dropout 

### Performance Metrics And Accuracy

In [60]:
y_pred=model.predict_classes(X_test)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [61]:
from sklearn.metrics import confusion_matrix

In [62]:
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 233, 2383]], dtype=int64)

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9113504556752279

In [64]:
#DRoupout Model
fom tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [55]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 25, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
